In [1]:
# 기본
import numpy as np
from numpy.linalg import inv
from numpy.linalg import solve
import random
import pandas as pd
from pandas import Series, DataFrame
import pandas_profiling
import matplotlib.pyplot as plt
import urllib
import matplotlib.dates as mdates
from matplotlib.dates import bytespdate2num
import matplotlib.ticker as ticker
from matplotlib import font_manager, rc
from matplotlib import style
import itertools
import seaborn as sns
import re
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np
import math
    
import matplotlib.pyplot as plt 
import matplotlib as mpl
import matplotlib.cm as cm
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.datasets import make_blobs

import ipywidgets as widgets
from IPython.display import display
from IPython.display import set_matplotlib_formats


# 시계열 데이터 처리
import calendar
import dateutil
from dateutil.parser import parse
import datetime
import statistics

# Network 분석
import networkx as nx

# 지도시각화
import folium
from folium import plugins
import html
import json
import geopy
from geopy.geocoders import Nominatim
import os
import requests
#import ipywidgets
#from IPython.display import Image
#from ipywidgets import interact


import collections
from collections import Counter
#from wordcloud import WordCloud, STOPWORDS as stopwords
#from PIL import Image, ImageFilter
#from wordcloud import ImageColorGenerator

# Network analysis
import networkx as nx
from tqdm import tqdm
import time

# from zeep import Client

from collections import namedtuple
import sqlite3


# 한 번에 matplotlib 그림 띄우기
%matplotlib inline    
%config InlineBackend.figure_format = 'retina'  #%matplotlib 뒤에 써주면 그래프를 더 높은 해상도로 보여줌

# 설정 관련
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# 우선 데이터 불러주기

In [108]:
data

,주문자ID,평균 주문금액(median),평균구매주기,전화주문고객,누적결제금액,19이후_누적주문횟수,군집_5,위도,경도
0,xigol80,121440.0,37,0.0,3186190,51.0,1,37.517540,126.934531
1,kyen1311,91620.0,75,0.0,626520,8.0,0,36.595609,127.466683
2,85643404@n,75820.0,14,0.0,3006060,65.0,1,36.338213,127.449423
3,ywca897,174080.0,64,0.0,2006080,23.0,3,37.484391,127.112913
4,pinkbabi100,94100.0,39,0.0,1851270,30.0,3,37.654079,127.241876
...,...,...,...,...,...,...,...,...,...
1457,clcc,91270.0,58,0.0,338450,4.0,0,37.597262,126.919697
1458,pp1664,78335.0,5,0.0,362890,5.0,0,37.551449,126.865809
1459,90873546@n,76745.0,5,0.0,560040,7.0,0,37.387484,127.114891
1460,104005455@n,82250.0,1,0.0,374870,9.0,0,37.255057,127.038831


In [111]:
data['전화주문고객'] = data['주문자ID'].str[:2] == 'pp'
data['전화주문고객'] = data['전화주문고객'].replace(True,1)
data['전화주문고객'] = data['전화주문고객'].replace(False,0)

,전화주문고객
군집_5,
0,349.0
1,24.0
2,2.0
3,102.0
4,5.0


In [3]:
pck_original = pd.read_csv('data/(Main)2019이후_3회_이상_주문한_사람들.csv', encoding='euc-kr')
data = pd.read_csv('data/clustering완료.csv', encoding='euc-kr')

In [4]:
pck_original

,주문번호,주문자ID,주문경로(PC/모바일),결제수단,총 결제금액,주문상품명,상품번호,주문횟수,상품명(패커스(PACKUS)),수량,분류,세부분류,사이즈,색상,개수,공급사,자체상품코드,주문일시,결제일시(입금확인일),발주일,배송대기일,배송시작일,배송완료일,세금구분,공급원가,상품 매입가,판매가,상품구매금액,상품별 추가할인 상세,옵션+판매가,쿠폰 할인금액,주문자 가입일,특별관리회원,회원추가항목_가입경로,회원추가항목_업종,주문 시 회원등급,주문자 주소,수령인 주소,위도,경도,주문-배송완료,발주일-배송시작일,배송시작-배송완료,회원가입연도,회원가입월,회원가입일,가장최근주문,19년이후 누적주문횟수,가입기간(일수),누적주문,주문연도.월,총주문량,세부분류사이즈
0,20170323-0000017,xigol80,PC,"쿠폰,카드",169500,"70파이 다용도컵 세트 1호 소 70ml 백색 PS 3,000개 1set(2박스)",38,477.0,"70파이 다용도컵 세트 1호 소 70ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,70ml,백색,6000.0,패커스,P00000BL,2017-03-23 03:09:18.000,2017-03-23 03:09:18.000,2017-03-23 03:09:18.000,2017-03-23 10:32:06.096,2017-03-23 18:43:40.000,2017-04-01 19:49:03.000,과세상품,42900.0,40500,53100,53100.0,NaN,53100.0,2000.0,2017-03-08 00:00:00.000,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,9 days 16:39:45.000000000,0 days 15:34:22.000000000,9 days 01:05:23.000000000,2017,3,3,2019-05-03 04:09:24.000,51.0,1004,22.0,2017_3,6000.0,다용도컵_70ml
1,20170323-0000017,xigol80,PC,"쿠폰,카드",169500,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",42,471.0,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",2,30.0,다용도컵,200ml,백색,2000.0,패커스,P00000BP,2017-03-23 03:09:18.000,2017-03-23 03:09:18.000,2017-03-23 03:09:18.000,2017-03-23 10:33:14.195,2017-03-23 18:43:40.000,2017-04-01 19:49:03.000,과세상품,44000.0,43000,59200,118400.0,NaN,59200.0,2000.0,2017-03-08 00:00:00.000,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,9 days 16:39:45.000000000,0 days 15:34:22.000000000,9 days 01:05:23.000000000,2017,3,3,2019-05-03 04:09:24.000,51.0,1004,22.0,2017_3,4000.0,다용도컵_200ml
2,20170421-0000015,xigol80,PC,"쿠폰,카드",113400,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",42,471.0,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",2,30.0,다용도컵,200ml,백색,2000.0,패커스,P00000BP,2017-04-21 03:38:14.000,2017-04-21 03:38:14.000,2017-04-21 03:38:14.000,2017-04-21 17:05:22.809,2017-04-21 17:10:58.000,2017-04-25 08:23:56.000,과세상품,44000.0,43000,59200,118400.0,NaN,59200.0,5000.0,2017-03-08 00:00:00.000,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,4 days 04:45:42.000000000,0 days 13:32:44.000000000,3 days 15:12:58.000000000,2017,3,3,2019-05-03 04:09:24.000,51.0,1004,22.0,2017_4,4000.0,다용도컵_200ml
3,20170519-0000012,xigol80,PC,"쿠폰,카드",208590,"70파이 다용도컵 세트 1호 특소 50ml 백색 PS 3,000개 1set(2박스)",47,274.0,"70파이 다용도컵 세트 1호 특소 50ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,50ml,백색,6000.0,패커스,P00000BU,2017-05-19 05:59:27.000,2017-05-19 05:59:27.000,2017-05-19 05:59:27.000,2017-05-19 16:01:20.590,2017-05-19 19:50:23.000,2017-05-27 05:35:19.000,과세상품,41250.0,40500,53100,53100.0,다용도 10%할인,53100.0,3000.0,2017-03-08 00:00:00.000,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,7 days 23:35:52.000000000,0 days 13:50:56.000000000,7 days 09:44:56.000000000,2017,3,3,2019-05-03 04:09:24.000,51.0,1004,22.0,2017_5,6000.0,다용도컵_50ml
4,20170519-0000012,xigol80,PC,"쿠폰,카드",208590,"70파이 다용도컵 세트 2호 대 80ml 백색 PS 3,000개 1set(2박스)",39,94.0,"70파이 다용도컵 세트 2호 대 80ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,80ml,백색,6000.0,패커스,P00000BM,2017-05-19 05:59:27.000,2017-05-19 05:59:27.000,2017-05-19 05:59:27.000,2017-05-19 16:01:20.590,2017-05-19 19:50:23.000,2017-05-27 05:35:19.000,과세상품,49500.0,45000,63600,63600.0,다용도 10%할인,63600.0,3000.0,2017-03-08 00:00:00.000,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,7 days 23:35:52.000000000,0 days 13:50:56.000000000,7 days 09:44:56.000000000,2017,3,3,2019-05-03 04:09:24.000,51.0,1004,22.0,2017_5,6000.0,다용도컵_80ml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33199,1908301422425566,q9691445,NaN,무통장 입금,54850,195파이 탕용기 소 300개 1set(2박스),1077,1366.0,195파이 탕용기 소 300개 1set(2박스),1,1.0,탕용기,195_小,NaN,600.0,패커스,P0000BPK,2019-08-30 14:22:42.000,2019-09-02 09:03:36.000,NaN,NaN,NaN,NaN,NaN,NaN,44550,57850,NaN,NaN,NaN,Na

In [5]:
data

,주문자ID,평균 주문금액(median),평균구매주기,전화주문고객,누적결제금액,19이후_누적주문횟수,군집_5,군집_4
0,xigol80,121440.0,37,0.0,3186190,51.0,1,2
1,kyen1311,91620.0,75,0.0,626520,8.0,0,0
2,85643404@n,75820.0,14,0.0,3006060,65.0,1,2
3,ywca897,174080.0,64,0.0,2006080,23.0,3,2
4,pinkbabi100,94100.0,39,0.0,1851270,30.0,3,2
...,...,...,...,...,...,...,...,...
1457,clcc,91270.0,58,0.0,338450,4.0,0,0
1458,pp1664,78335.0,5,0.0,362890,5.0,0,0
1459,90873546@n,76745.0,5,0.0,560040,7.0,0,0
1460,104005455@n,82250.0,1,0.0,374870,9.0,0,0


# 위도, 경도와 clustering한 파일 합쳐주기

In [9]:
cst_lon_lat = pck_original[['주문자ID','위도','경도']].drop_duplicates(['주문자ID'])

In [10]:
cst_lon_lat

,주문자ID,위도,경도
0,xigol80,37.517540,126.934531
17,kyen1311,36.595609,127.466683
21,85643404@n,36.338213,127.449423
25,ywca897,37.484391,127.112913
33,pinkbabi100,37.654079,127.241876
...,...,...,...
32995,clcc,37.597262,126.919697
32998,pp1664,37.551449,126.865809
33007,90873546@n,37.387484,127.114891
33024,104005455@n,37.255057,127.038831


In [11]:
data = data.merge(cst_lon_lat, how='left', on='주문자ID')

In [17]:
del data['군집_4']
data

,주문자ID,평균 주문금액(median),평균구매주기,전화주문고객,누적결제금액,19이후_누적주문횟수,군집_5,위도,경도
0,xigol80,121440.0,37,0.0,3186190,51.0,1,37.517540,126.934531
1,kyen1311,91620.0,75,0.0,626520,8.0,0,36.595609,127.466683
2,85643404@n,75820.0,14,0.0,3006060,65.0,1,36.338213,127.449423
3,ywca897,174080.0,64,0.0,2006080,23.0,3,37.484391,127.112913
4,pinkbabi100,94100.0,39,0.0,1851270,30.0,3,37.654079,127.241876
...,...,...,...,...,...,...,...,...,...
1457,clcc,91270.0,58,0.0,338450,4.0,0,37.597262,126.919697
1458,pp1664,78335.0,5,0.0,362890,5.0,0,37.551449,126.865809
1459,90873546@n,76745.0,5,0.0,560040,7.0,0,37.387484,127.114891
1460,104005455@n,82250.0,1,0.0,374870,9.0,0,37.255057,127.038831


# 지도에 뿌려보기

## 우선 군집별로 data 나눠주기

In [20]:
# 군집 0
cluster_0 = data[data['군집_5']==0]
# 군집 1
cluster_1 = data[data['군집_5']==1]
# 군집 2
cluster_2 = data[data['군집_5']==2]
# 군집 3
cluster_3 = data[data['군집_5']==3]
# 군집 4
cluster_4 = data[data['군집_5']==4]

## 지도에 각 군집들 뿌리기

In [102]:
korea = [36.39, 127.28]
map_korea = folium.Map(location=korea, zoom_start=7)

In [22]:
cluster_0

,주문자ID,평균 주문금액(median),평균구매주기,전화주문고객,누적결제금액,19이후_누적주문횟수,군집_5,위도,경도
1,kyen1311,91620.0,75,0.0,626520,8.0,0,36.595609,127.466683
6,rudals730,117040.0,48,0.0,764190,17.0,0,36.629524,127.489628
7,39618168@n,35210.0,26,0.0,345550,14.0,0,37.293246,127.454631
11,rlawltnr85,52995.0,53,0.0,463880,12.0,0,35.138261,126.893019
14,love1203a,111295.0,100,0.0,1010180,12.0,0,37.506309,126.936807
...,...,...,...,...,...,...,...,...,...
1457,clcc,91270.0,58,0.0,338450,4.0,0,37.597262,126.919697
1458,pp1664,78335.0,5,0.0,362890,5.0,0,37.551449,126.865809
1459,90873546@n,76745.0,5,0.0,560040,7.0,0,37.387484,127.114891
1460,104005455@n,82250.0,1,0.0,374870,9.0,0,37.255057,127.038831


In [54]:
map_korea

In [97]:
# 군집 0 찍기
for customer in tqdm(list(cluster_0['주문자ID'].unique())):
    temp = cluster_0[cluster_0['주문자ID']==customer]
    folium.Marker([temp['위도'].values[0], temp['경도'].values[0]], popup=customer, icon=folium.Icon(icon='cloud', color = "black")).add_to(map_korea)

# 군집 1 찍기
for customer in tqdm(list(cluster_1['주문자ID'].unique())):
    temp = cluster_1[cluster_1['주문자ID']==customer]
    folium.Marker([temp['위도'].values[0], temp['경도'].values[0]], popup=customer, icon=folium.Icon(icon='cloud', color = "green")).add_to(map_korea)
    
# 군집 2 찍기
for customer in tqdm(list(cluster_2['주문자ID'].unique())):
    temp = cluster_2[cluster_2['주문자ID']==customer]
    folium.Marker([temp['위도'].values[0], temp['경도'].values[0]], popup=customer, icon=folium.Icon(icon='cloud', color = "purple")).add_to(map_korea)
    
# 군집 3 찍기
for customer in tqdm(list(cluster_3['주문자ID'].unique())):
    temp = cluster_3[cluster_3['주문자ID']==customer]
    folium.Marker([temp['위도'].values[0], temp['경도'].values[0]], popup=customer, icon=folium.Icon(icon='cloud', color = 'blue')).add_to(map_korea)
    
# 군집 4 찍기
for customer in tqdm(list(cluster_4['주문자ID'].unique())):
    temp = cluster_4[cluster_4['주문자ID']==customer]
    folium.Marker([temp['위도'].values[0], temp['경도'].values[0]], popup=customer, icon=folium.Icon(icon='cloud', color = "red")).add_to(map_korea)
    

    

100%|█████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 501.33it/s]


In [94]:
map_korea

In [104]:
map_korea.save('지도시각화_프렌즈 등급 이상.html')